In [ ]:
def benchmark(func):
    import time
    def wrapper(*args, **kwargs):
        started = time.time()
        res = func(*args, **kwargs)
        print('running for', time.time() - started)
        return res
    return wrapper

def retry_on_error(exception: Exception, tries: int = 3):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(tries):
                try:
                    raise Exception
                    return func(*args, **kwargs)
                except exception:
                    print('exception...')
        return wrapper
    return decorator

In [53]:
from collections import deque

class Memoize:
    
    def __init__(self, cache_size=5):
        self.cache_size = cache_size
        self.call_args_queue = deque()
        self.call_args_to_result = {}
        
    @staticmethod
    def _hash_args(args, kwargs):
        return hash(str(args) + str(kwargs))
    
    def _clean_queue(self):
        if len(self.call_args_to_result) < self.cache_size:
            return
        popped = self.call_args_queue.popleft()
        print('we clean from popped')
        del self.call_args_to_result[popped]
        
        
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            hashed_args = Memoize._hash_args(args, kwargs)
            print('hashed', hashed_args)
            if not hashed_args in self.call_args_to_result:
                print('save')
                self.call_args_to_result[hashed_args] = func(*args, **kwargs)
                self._clean_queue()
                self.call_args_queue.append(hashed_args)
                return self.call_args_to_result[hashed_args]
            print('use mem')
            return self.call_args_to_result[hashed_args]
        return wrapper
            

In [57]:
@Memoize(5)
def f(a, b):
    return a,b 

f(34, 2)
f(34,2)
f(3,2)
f(3,3)
f(3,4)
f(3,5)

hashed 3858993327405094944
save
hashed 3858993327405094944
use mem
hashed -7763230149830122360
save
hashed 893375603462835068
save
hashed -1895409055782295232
save
hashed 4853616770151224157
save
we clean from popped


(3, 5)